In [3]:
import sys
sys.path.append('../src/')

from benchmark.tools import *
from benchmark.models import Detector
from benchmark.utils import TsvHandle, run_detection

import wider_dataset
# https://github.com/borhanMorphy/fastface
import fastface as ff

In [4]:
train, test, validation = tfds.load('wider_dataset', split=['train', 'test', 'validation'])
data = validation

In [5]:
class FastFaceDetector(Detector):
    def __init__(self):
        model_weights = "./lffd_original.ckpt"
        self.detector = ff.FaceDetector.from_pretrained("./lffd_original.ckpt")
        self.detector.summarize()
    
    def detect(self, image):
        try:
            predicted, = self.detector.predict(image, det_threshold=.8, iou_threshold=.4)
            return predicted['boxes'], predicted['scores']
        except Exception:
            return [], []
    
fast_face = FastFaceDetector()

D:\anaconda3\envs\fastface2\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

  | Name | Type | Params
------------------------------
0 | arch | LFFD | 2.3 M 
------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.158     Total estimated model params size (MB)


In [6]:
with TsvHandle('fastface_filtered') as tsv:
    run_detection(tsv, data, fast_face, None, False, False, None, 2000)

Running detection
[====================] 100% [3226/3226]